# Jaundice Detection Project

For previous dataset we will be using skin detections and putting it in dataloader but not in new data set

## Skin Detection

In [ ]:
from code_snippets import skin_detection
custom_skin_detector = skin_detection.custom_skin_detector

## Getting the data

In [ ]:
# import torch
# from torch import nn
# from torch.utils.data import DataLoader, random_split
# from torchvision import datasets, transforms
# import numpy as np

# # transformations
# transform = transforms.Compose([
#     SkinTransform(skin_detector = custom_skin_detector),
#     transforms.Resize((512,512)),    
#     transforms.ToTensor()
# ])

# # Load the datasets
# dataset = datasets.ImageFolder('./images',transform=transform)


# # combined dataset
# train_size = int(0.8*len(dataset))
# test_size = len(dataset) - train_size

# train_dataset, test_dataset = random_split(dataset, [train_size,test_size],generator=torch.Generator().manual_seed(42))


# # Create the dataloaders
# batch_size = 32
# train_loader = DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
# import matplotlib.pyplot as plt
# image,labels = next(iter(train_loader))
# plt.figure()
# plt.imshow(image[0].permute(1,2,0).numpy())
# plt.show()

New Dataset

In [ ]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets, transforms
import numpy as np
import PIL
from PIL import Image, ImageFilter
import os

jaundice_csv = pd.read_csv("./NeoJaundice/chd_jaundice_published_2.csv")
jaundice_csv.head()

In [ ]:
jaundice_csv.loc[1024]

#### Putting it all together

In [ ]:
from code_snippets import custom_dataset
from code_snippets.image_filters import dialation_filter, erosion_filter, median_filter
transform = transforms.Compose([
    transforms.CenterCrop(150),
    transforms.Resize((224,224)),
    # median_filter.MedianFilter(kernel_size=5),
    dialation_filter.Dilation(kernel_size=5),
    erosion_filter.Erosion(kernel_size=5),
    transforms.ToTensor(),
])

# importing dataset using new customdataset class
dataset = custom_dataset.CustomDataset("./NeoJaundice/chd_jaundice_published_2.csv", "./NeoJaundice/images/", "image_idx", "Treatment",transform=transform)

# splitting size
train_size = int(0.8*len(dataset))
test_size = len(dataset) - train_size

# split the data
train_dataset, test_dataset = random_split(dataset, [train_size,test_size],generator=torch.Generator().manual_seed(42))

# Create the dataloaders
batch_size = 1
train_loader = DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (20,20))
for i in range(16):
    fig.add_subplot(4,4,i+1)
    image,labels = next(iter(train_loader))
    plt.imshow(image[0].permute(1,2,0).numpy())
    plt.title(labels[0].item())
plt.show()


## Models

In [ ]:
import code_snippets.models

from torchvision import models

device = "cuda" if torch.cuda.is_available() else "cpu"


model = code_snippets.models.alexnet(device)
model = models.alexnet(pretrained = True)
model.eval()
with torch.no_grad():
    nn_filters = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(nn_filters,1)
    model = model.to(device)

## optimizer and loss functions

In [ ]:
optimizer_fn = torch.optim.SGD(model.parameters(),lr=0.003,weight_decay=0.000003)
loss_fn = nn.BCEWithLogitsLoss()

## Train and Test loops

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct/len(y_pred)) *100
    return acc

In [ ]:
# Training loop
from torcheval.metrics.aggregation.auc import AUC
from tqdm import tqdm

torch.manual_seed(42)
torch.cuda.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

auc = AUC()
auc_test = AUC()
for epoch in range(5):
    print(f'!!epoch {epoch+1} is running!!')

    cum_loss = 0
    
    for i,(images, labels) in tqdm(enumerate(train_loader),total = len(train_loader)):
        # Forward pass
        model.train()
        images = images.to(device)
        labels = labels.unsqueeze(1).type(torch.float32)
        labels= labels.to(device)

        output_logits = model(images)
        output_pred = torch.round(torch.sigmoid(output_logits))
        acc = (output_pred == labels).sum().item() / labels.size(0)
        
        auc.update(labels, output_pred)

        loss = loss_fn(output_logits,labels)
        cum_loss += loss
        
        # Backward pass and optimization
        optimizer_fn.zero_grad()
        loss.backward()
        optimizer_fn.step()
    print(f'Epoch {epoch+1}, Train Loss: {cum_loss/len(train_loader)}, Accuracy: {acc}, Auc: {auc.compute()}')
    # auc.reset()

    # test loop
    model.eval()
    with torch.no_grad():
        cum_loss = 0
        cum_acc = 0
        for images_test,labels_test in test_loader:
            images_test = images_test.to(device)
            labels_test = labels_test.unsqueeze(1).type(torch.float32)
            labels_test = labels_test.to(device)

            test_logits = model(images_test)
            test_pred = torch.round(torch.sigmoid(test_logits))
            test_loss = loss_fn(test_logits,labels_test)
            auc_test.update(test_pred, labels_test)
            cum_loss += test_loss
            cum_acc += accuracy_fn(y_true = labels_test, y_pred = test_pred)
        print(f'Epoch {epoch+1}, Test Loss {cum_loss/len(test_loader)}, accuracy {cum_acc/len(test_loader)}, Auc: {auc_test.compute()}')
        # auc_test.reset()

In [ ]:
import matplotlib.pyplot as plt
image,labels = next(iter(train_loader))
plt.figure()
plt.imshow(image[0].permute(1,2,0).numpy())
plt.show()

In [ ]:
from ignite.metrics import ClassificationReport, ConfusionMatrix
